In [62]:
import pandas as pd

from surprise import SVD, SVDpp
from surprise import NormalPredictor

from surprise import Dataset
from surprise import Reader

from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

from helpers import *

In [56]:
data = load_data("data/data_train.csv")
data.head()

,user,movie,rating
0,r44,c1,4
1,r61,c1,3
2,r67,c1,4
3,r72,c1,3
4,r86,c1,5


In [57]:
dataset = Dataset.load_from_df(data[['user', 'movie', 'rating']], Reader(rating_scale=(1, 5)))
# cross_validate(NormalPredictor(), dataset, cv=5)

In [70]:
param_grid = {'n_epochs': [10, 30], 'lr_all': [0.0005, 0.003],
              'reg_all': [0.1, 0.4]}
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(dataset)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.0033560111589688
{'n_epochs': 30, 'lr_all': 0.0005, 'reg_all': 0.1}


In [71]:
algo = gs.best_estimator['rmse']

# Run 5-fold cross-validation and print results.
cross_validate(algo, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0018  0.9997  1.0019  1.0022  1.0048  1.0021  0.0016  
MAE (testset)     0.8138  0.8126  0.8136  0.8130  0.8153  0.8136  0.0009  
Fit time          2366.12 2354.55 2348.27 2457.26 2464.48 2398.14 51.59   
Test time         37.80   35.44   36.06   42.19   41.96   38.69   2.87    


{'test_rmse': array([1.00182325, 0.99969704, 1.00187482, 1.00215067, 1.00484139]),
 'test_mae': array([0.81377047, 0.81260757, 0.81360828, 0.81297216, 0.81527127]),
 'fit_time': (2366.118020057678,
  2354.547372817993,
  2348.273766040802,
  2457.263636112213,
  2464.4815440177917),
 'test_time': (37.79636311531067,
  35.43850111961365,
  36.0573787689209,
  42.187068939208984,
  41.95561122894287)}

In [72]:
gen_submission(r'data/submission3.csv', algo)